In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import mutual_info_classif,chi2
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import roc_auc_score, mean_squared_error

# from imblearn.over_sampling import SMOTE
# from imblearn.under_sampling import CondensedNearestNeighbour

# import category_encoders as ce

plt.style.use('seaborn-colorblind')
%matplotlib inline

In [144]:
data = pd.read_csv('data/adult.data')
data.columns = [
    'age','workclass','fnlwgt','education','education-num','marital-status',
    'occupation','relationship','race','sex','capital-gain','capital-loss',
    'hours-per-week','native-country','target'
]
print(data.shape)
data.head(8)

(32560, 15)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
5,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
6,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
7,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K


## Пропущенные значения

Проверка пропущенных значений

In [145]:
def check_missing(data,output_path=None):    
    result = pd.concat([data.isnull().sum(),data.isnull().mean()],axis=1)
    result = result.rename(index=str,columns={0:'total missing',1:'proportion'})
    if output_path is not None:
        result.to_csv(output_path+'missing.csv')
        print(output_path, 'missing.csv')
    return result

In [146]:
check_missing(data=data)

,total missing,proportion
age,0,0.0
workclass,0,0.0
fnlwgt,0,0.0
education,0,0.0
education-num,0,0.0
marital-status,0,0.0
occupation,0,0.0
relationship,0,0.0
race,0,0.0
sex,0,0.0


In [150]:
hoursperweek_missing_indexes = data.sample(frac=0.1).index
age_missing_indexes = data.sample(frac=0.15).index

data_hoursperweek, data_age = data['hours-per-week'].copy(), data['age'].copy()
data_hoursperweek[hoursperweek_missing_indexes], data_age[age_missing_indexes] = np.NaN, np.NaN

data['hours-per-week'] = data_hoursperweek
data['age'] = data_age

In [151]:
check_missing(data=data)

,total missing,proportion
age,4884,0.15
workclass,0,0.00
fnlwgt,0,0.00
education,0,0.00
education-num,0,0.00
marital-status,0,0.00
occupation,0,0.00
relationship,0,0.00
race,0,0.00
sex,0,0.00


Удаление пропущенных значений

In [153]:
def drop_missing(data,axis=0):
    data_copy = data.copy(deep=True)
    data_copy = data_copy.dropna(axis=axis,inplace=False)
    return data_copy

In [154]:
data2 = drop_missing(data=data)
data2.shape

(24924, 15)

Добавление переменной оценки пропущенных значений

In [155]:
def add_var_denote_NA(data,NA_col=[]):
    data_copy = data.copy(deep=True)
    for i in NA_col:
        if data_copy[i].isnull().sum()>0:
            data_copy[i+'_is_NA'] = np.where(data_copy[i].isnull(),1,0)
        else:
            warn("Нет пропущенных значений" % i)          
    return data_copy

In [156]:
col = 'age'
data3 = add_var_denote_NA(data=data,NA_col=[col])
print(data3[col].value_counts())
data3.head(8)

36.0    774
31.0    767
23.0    754
33.0    754
35.0    747
28.0    746
37.0    746
30.0    728
34.0    722
25.0    718
41.0    704
27.0    694
38.0    693
29.0    689
39.0    685
24.0    685
32.0    681
26.0    675
42.0    670
22.0    669
40.0    668
20.0    656
43.0    652
44.0    627
21.0    620
45.0    614
19.0    606
47.0    601
46.0    599
50.0    502
       ... 
60.0    266
61.0    266
62.0    220
63.0    199
64.0    190
65.0    157
66.0    127
67.0    124
68.0    104
69.0     89
70.0     70
73.0     58
71.0     55
72.0     52
74.0     42
75.0     41
76.0     37
90.0     36
77.0     25
79.0     19
80.0     19
78.0     19
81.0     17
82.0     12
84.0      8
83.0      5
85.0      3
88.0      3
86.0      1
87.0      1
Name: age, Length: 73, dtype: int64


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,age_is_NA
0,50.0,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13.0,United-States,<=50K,0
1,38.0,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,NaN,United-States,<=50K,0
2,53.0,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40.0,United-States,<=50K,0
3,28.0,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40.0,Cuba,<=50K,0
4,37.0,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40.0,United-States,<=50K,0
5,49.0,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16.0,Jamaica,<=50K,0
6,52.0,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45.0,United-States,>50K,0
7,31.0,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50.0,United-States,>50K,0


Заполнение пропусков выборочным значением

In [157]:
def impute_NA_with_arbitrary(data,impute_value,NA_col=[]):  
    data_copy = data.copy(deep=True)
    for i in NA_col:
        if data_copy[i].isnull().sum()>0:
            data_copy[i+'_'+str(impute_value)] = data_copy[i].fillna(impute_value)
        else:
            warn("Нет пропущенных значений" % i)
    return data_copy

In [158]:
col = 'age'
data4 = impute_NA_with_arbitrary(data=data,impute_value=-999,NA_col=[col])
data4[data4.age.isna()][['age','age_-999']].head(8)

,age,age_-999
15,NaN,-999.0
17,NaN,-999.0
24,NaN,-999.0
30,NaN,-999.0
32,NaN,-999.0
37,NaN,-999.0
56,NaN,-999.0
59,NaN,-999.0


Заполнение пропущенных значений средним/медианой/модой

In [159]:
def impute_NA_with_avg(data,strategy='mean',NA_col=[]):
    data_copy = data.copy(deep=True)
    for i in NA_col:
        if data_copy[i].isnull().sum()>0:
            if strategy=='mean':
                data_copy[i+'_impute_mean'] = data_copy[i].fillna(data[i].mean())
            elif strategy=='median':
                data_copy[i+'_impute_median'] = data_copy[i].fillna(data[i].median())
            elif strategy=='mode':
                data_copy[i+'_impute_mode'] = data_copy[i].fillna(data[i].mode()[0])
        else:
            warn("Нет пропущенных значений" % i)
    return data_copy  

In [160]:
cal = 'age'
print(data.age.median())
data5 = impute_NA_with_avg(data=data,strategy='median',NA_col=[col])
data5.head(8)

37.0


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,age_impute_median
0,50.0,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13.0,United-States,<=50K,50.0
1,38.0,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,NaN,United-States,<=50K,38.0
2,53.0,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40.0,United-States,<=50K,53.0
3,28.0,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40.0,Cuba,<=50K,28.0
4,37.0,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40.0,United-States,<=50K,37.0
5,49.0,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16.0,Jamaica,<=50K,49.0
6,52.0,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45.0,United-States,>50K,52.0
7,31.0,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50.0,United-States,>50K,31.0


Заполнение пропусков значением из "хвоста" распределения

In [161]:
def impute_NA_with_end_of_distribution(data,NA_col=[]):
    data_copy = data.copy(deep=True)
    for i in NA_col:
        if data_copy[i].isnull().sum()>0:
            data_copy[i+'_impute_end_of_distri'] = data_copy[i].fillna(data[i].mean()+3*data[i].std())
        else:
            warn("Нет пропущенных значений" % i)
    return data_copy     

In [162]:
data6 = impute_NA_with_end_of_distribution(data=data,NA_col=['age'])
data6.head(8)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,age_impute_end_of_distri
0,50.0,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13.0,United-States,<=50K,50.0
1,38.0,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,NaN,United-States,<=50K,38.0
2,53.0,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40.0,United-States,<=50K,53.0
3,28.0,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40.0,Cuba,<=50K,28.0
4,37.0,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40.0,United-States,<=50K,37.0
5,49.0,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16.0,Jamaica,<=50K,49.0
6,52.0,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45.0,United-States,>50K,52.0
7,31.0,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50.0,United-States,>50K,31.0


Заполнение пропусков случайными значениями

In [163]:
def impute_NA_with_random(data,NA_col=[],random_state=0):
    data_copy = data.copy(deep=True)
    for i in NA_col:
        if data_copy[i].isnull().sum()>0:
            data_copy[i+'_random'] = data_copy[i]
            random_sample = data_copy[i].dropna().sample(data_copy[i].isnull().sum(), random_state=random_state)
            random_sample.index = data_copy[data_copy[i].isnull()].index
            data_copy.loc[data_copy[i].isnull(), str(i)+'_random'] = random_sample
        else:
            warn("Нет пропущенных значений" % i)
    return data_copy 

In [164]:
data7 = impute_NA_with_random(data=data,NA_col=['age'])
data7.head(8)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,age_random
0,50.0,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13.0,United-States,<=50K,50.0
1,38.0,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,NaN,United-States,<=50K,38.0
2,53.0,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40.0,United-States,<=50K,53.0
3,28.0,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40.0,Cuba,<=50K,28.0
4,37.0,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40.0,United-States,<=50K,37.0
5,49.0,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16.0,Jamaica,<=50K,49.0
6,52.0,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45.0,United-States,>50K,52.0
7,31.0,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50.0,United-States,>50K,31.0


## Выбросы

Детекция с помощью выборочных значений

In [165]:
def outlier_detect_arbitrary(data,col,upper_fence,lower_fence):
    para = (upper_fence, lower_fence)
    tmp = pd.concat([data[col]>upper_fence,data[col]<lower_fence],axis=1)
    outlier_index = tmp.any(axis=1)
    print('Количество выбросов в данных:',outlier_index.value_counts()[1])
    print('Доля выбросов:',outlier_index.value_counts()[1]/len(outlier_index))    
    return outlier_index, para

In [166]:
index,para = outlier_detect_arbitrary(data=data,col='hours-per-week',upper_fence=100,lower_fence=5)
print('Верхняя граница:',para[0],'\nНижняя граница:',para[1])

Количество выбросов в данных: 131
Доля выбросов: 0.0040233415233415235
Верхняя граница: 100 
Нижняя граница: 5


In [167]:
data.loc[index,'hours-per-week'].sort_values()

32524    1.0
5807     1.0
5765     1.0
5631     1.0
20071    1.0
19336    1.0
11450    1.0
20908    1.0
22959    1.0
19749    1.0
24283    1.0
23535    1.0
29751    1.0
8446     1.0
9146     1.0
1261     1.0
1035     1.0
188      1.0
25077    1.0
13986    2.0
14644    2.0
17633    2.0
13369    2.0
12917    2.0
11786    2.0
19352    2.0
20607    2.0
20562    2.0
22129    2.0
22386    2.0
        ... 
11021    4.0
10385    4.0
11730    4.0
9820     4.0
12067    4.0
12650    4.0
9339     4.0
13073    4.0
13262    4.0
13842    4.0
14546    4.0
8309     4.0
15191    4.0
23635    4.0
15622    4.0
17385    4.0
17408    4.0
8109     4.0
18231    4.0
18640    4.0
18835    4.0
6266     4.0
5439     4.0
4463     4.0
10714    4.0
22566    4.0
22727    4.0
22927    4.0
15642    4.0
15772    4.0
Name: hours-per-week, Length: 131, dtype: float64

Интерквартильное расстояние

In [180]:
def outlier_detect_IQR(data,col,threshold=3):    
    IQR = data[col].quantile(0.75) - data[col].quantile(0.25)
    Lower_fence = data[col].quantile(0.25) - (IQR * threshold)
    Upper_fence = data[col].quantile(0.75) + (IQR * threshold)
    para = (Upper_fence, Lower_fence)
    tmp = pd.concat([data[col]>Upper_fence,data[col]<Lower_fence],axis=1)
    outlier_index = tmp.any(axis=1)
    print('Количество выбросов в данных:',outlier_index.value_counts()[1])
    print('Доля выбросов:',outlier_index.value_counts()[1]/len(outlier_index))
    return outlier_index, para

In [181]:
index,para = outlier_detect_IQR(data=data,col='hours-per-week',threshold=5)
print('Верхняя граница:',para[0],'\nНижняя граница:',para[1])

Количество выбросов в данных: 1319
Доля выбросов: 0.04050982800982801
Верхняя граница: 70.0 
Нижняя граница: 15.0


In [182]:
data.loc[index,'hours-per-week'].sort_values()

19336     1.0
1035      1.0
20071     1.0
29751     1.0
20908     1.0
22959     1.0
9146      1.0
11450     1.0
23535     1.0
5631      1.0
5765      1.0
5807      1.0
19749     1.0
8446      1.0
32524     1.0
188       1.0
24283     1.0
25077     1.0
1261      1.0
1457      2.0
31389     2.0
10840     2.0
22129     2.0
13986     2.0
22386     2.0
2931      2.0
19426     2.0
3290      2.0
30199     2.0
5171      2.0
         ... 
3577     99.0
29988    99.0
30035    99.0
10142    99.0
30609    99.0
4085     99.0
30753    99.0
21055    99.0
4440     99.0
10985    99.0
13857    99.0
19052    99.0
23178    99.0
29750    99.0
9830     99.0
8071     99.0
24482    99.0
1886     99.0
30991    99.0
13616    99.0
13579    99.0
19398    99.0
13557    99.0
23225    99.0
10265    99.0
27221    99.0
19528    99.0
30765    99.0
18602    99.0
8392     99.0
Name: hours-per-week, Length: 1319, dtype: float64

Среднее-среднеквадратичное отклонение

In [183]:
def outlier_detect_mean_std(data,col,threshold=3):
    Upper_fence = data[col].mean() + threshold * data[col].std()
    Lower_fence = data[col].mean() - threshold * data[col].std()   
    para = (Upper_fence, Lower_fence)   
    tmp = pd.concat([data[col]>Upper_fence,data[col]<Lower_fence],axis=1)
    outlier_index = tmp.any(axis=1)
    print('Количество выбросов в данных:',outlier_index.value_counts()[1])
    print('Доля выбросов:',outlier_index.value_counts()[1]/len(outlier_index))
    return outlier_index, para

In [184]:
index,para = outlier_detect_mean_std(data=data,col='hours-per-week',threshold=3)
print('Верхняя граница:',para[0],'\nНижняя граница:',para[1])

Количество выбросов в данных: 400
Доля выбросов: 0.012285012285012284
Верхняя граница: 77.43464264863707 
Нижняя граница: 3.4227147087202923


In [185]:
data.loc[index,'hours-per-week'].sort_values()

23535     1.0
20908     1.0
9146      1.0
11450     1.0
1261      1.0
22959     1.0
24283     1.0
5807      1.0
5765      1.0
1035      1.0
19336     1.0
5631      1.0
8446      1.0
20071     1.0
29751     1.0
188       1.0
25077     1.0
19749     1.0
32524     1.0
11786     2.0
2931      2.0
6845      2.0
6179      2.0
3290      2.0
23247     2.0
12917     2.0
14644     2.0
13986     2.0
29866     2.0
22129     2.0
         ... 
26857    99.0
26637    99.0
26595    99.0
26249    99.0
4440     99.0
25985    99.0
25830    99.0
25805    99.0
25712    99.0
5375     99.0
25353    99.0
5431     99.0
24482    99.0
4307     99.0
23829    99.0
6617     99.0
23397    99.0
23225    99.0
12787    99.0
23178    99.0
22719    99.0
22556    99.0
22312    99.0
22215    99.0
21146    99.0
21055    99.0
8071     99.0
8392     99.0
23648    99.0
32530    99.0
Name: hours-per-week, Length: 400, dtype: float64

Медианы абсолютного отклонения (MAD) 

In [188]:
def outlier_detect_MAD(data,col,threshold=3.5):
    median = data[col].median()
    median_absolute_deviation = np.median([np.abs(y - median) for y in data[col]])
    modified_z_scores = pd.Series([0.6745 * (y - median) / median_absolute_deviation for y in data[col]])
    outlier_index = np.abs(modified_z_scores) > threshold
    print(outlier_index)
#     print('Количество выбросов в данных:',outlier_index.value_counts()[1])
#     print('Доля выбросов:',outlier_index.value_counts()[1]/len(outlier_index))
    return outlier_index

In [189]:
index = outlier_detect_MAD(data=data,col='hours-per-week',threshold=3.5)

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
32530    False
32531    False
32532    False
32533    False
32534    False
32535    False
32536    False
32537    False
32538    False
32539    False
32540    False
32541    False
32542    False
32543    False
32544    False
32545    False
32546    False
32547    False
32548    False
32549    False
32550    False
32551    False
32552    False
32553    False
32554    False
32555    False
32556    False
32557    False
32558    False
32559    False
Length: 32560, dtype: bool


/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:3405: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


Замена выброса выборочным значением

In [190]:
def impute_outlier_with_arbitrary(data,outlier_index,value,col=[]):
    data_copy = data.copy(deep=True)
    for i in col:
        data_copy.loc[outlier_index,i] = value
    return data_copy

In [191]:
data2 = impute_outlier_with_arbitrary(data=data,outlier_index=index,
                                         value=-999,col=['Fare'])
data2[25:35]

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,Fare
25,19.0,Private,168294,HS-grad,9,Never-married,Craft-repair,Own-child,White,Male,0,0,40.0,United-States,<=50K,NaN
26,54.0,?,180211,Some-college,10,Married-civ-spouse,?,Husband,Asian-Pac-Islander,Male,0,0,NaN,South,>50K,NaN
27,39.0,Private,367260,HS-grad,9,Divorced,Exec-managerial,Not-in-family,White,Male,0,0,80.0,United-States,<=50K,NaN
28,49.0,Private,193366,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40.0,United-States,<=50K,NaN
29,23.0,Local-gov,190709,Assoc-acdm,12,Never-married,Protective-serv,Not-in-family,White,Male,0,0,52.0,United-States,<=50K,NaN
30,NaN,Private,266015,Some-college,10,Never-married,Sales,Own-child,Black,Male,0,0,44.0,United-States,<=50K,NaN
31,45.0,Private,386940,Bachelors,13,Divorced,Exec-managerial,Own-child,White,Male,0,1408,NaN,United-States,<=50K,NaN
32,NaN,Federal-gov,59951,Some-college,10,Married-civ-spouse,Adm-clerical,Own-child,White,Male,0,0,NaN,United-States,<=50K,NaN
33,22.0,State-gov,311512,Some-college,10,Married-civ-spouse,Other-service,Husband,Black,Male,0,0,15.0,United-States,<=50K,NaN
34,48.0,Private,242406,11th,7,Never-married,Machine-op-inspct,Unmarried,White,Male,0,0,40.0,Puerto-Rico,<=50K,NaN


Виндзоризация

{92, 19, 101, 58, 1053, 91, 26, 78, 10, 13, −40, 101, 86, 85, 15, 89, 89, 28, −5, 41}  

{92, 19, 101, 58, 101, 91, 26, 78, 10, 13, −5, 101, 86, 85, 15, 89, 89, 28, −5, 41} 

In [192]:
def windsorization(data,col,para,strategy='both'):
    data_copy = data.copy(deep=True)  
    if strategy == 'both':
        data_copy.loc[data_copy[col]>para[0],col] = para[0]
        data_copy.loc[data_copy[col]<para[1],col] = para[1]
    elif strategy == 'top':
        data_copy.loc[data_copy[col]>para[0],col] = para[0]
    elif strategy == 'bottom':
        data_copy.loc[data_copy[col]<para[1],col] = para[1]  
    return data_copy

In [193]:
data3 = windsorization(data=data,col='hours-per-week',para=para,strategy='both')
data3[25:35]

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
25,19.0,Private,168294,HS-grad,9,Never-married,Craft-repair,Own-child,White,Male,0,0,40.000000,United-States,<=50K
26,54.0,?,180211,Some-college,10,Married-civ-spouse,?,Husband,Asian-Pac-Islander,Male,0,0,NaN,South,>50K
27,39.0,Private,367260,HS-grad,9,Divorced,Exec-managerial,Not-in-family,White,Male,0,0,77.434643,United-States,<=50K
28,49.0,Private,193366,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40.000000,United-States,<=50K
29,23.0,Local-gov,190709,Assoc-acdm,12,Never-married,Protective-serv,Not-in-family,White,Male,0,0,52.000000,United-States,<=50K
30,NaN,Private,266015,Some-college,10,Never-married,Sales,Own-child,Black,Male,0,0,44.000000,United-States,<=50K
31,45.0,Private,386940,Bachelors,13,Divorced,Exec-managerial,Own-child,White,Male,0,1408,NaN,United-States,<=50K
32,NaN,Federal-gov,59951,Some-college,10,Married-civ-spouse,Adm-clerical,Own-child,White,Male,0,0,NaN,United-States,<=50K
33,22.0,State-gov,311512,Some-college,10,Married-civ-spouse,Other-service,Husband,Black,Male,0,0,15.000000,United-States,<=50K
34,48.0,Private,242406,11th,7,Never-married,Machine-op-inspct,Unmarried,White,Male,0,0,40.000000,Puerto-Rico,<=50K


Удаление выбросов

In [194]:
def drop_outlier(data,outlier_index):
    data_copy = data[~outlier_index]
    return data_copy

In [196]:
data4 = drop_outlier(data=data,outlier_index=index)
print(data4['hours-per-week'].max())
print(data4['hours-per-week'].min())

99.0
1.0


Замена выбросов средним/медианой/модой

In [197]:
def impute_outlier_with_avg(data,col,outlier_index,strategy='mean'):
    data_copy = data.copy(deep=True)
    if strategy=='mean':
        data_copy.loc[outlier_index,col] = data_copy[col].mean()
    elif strategy=='median':
        data_copy.loc[outlier_index,col] = data_copy[col].median()
    elif strategy=='mode':
        data_copy.loc[outlier_index,col] = data_copy[col].mode()[0]           
    return data_copy

In [198]:
data5 = impute_outlier_with_avg(data=data,col='hours-per-week',
                                   outlier_index=index,strategy='mean')
data5[25:35]

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
25,19.0,Private,168294,HS-grad,9,Never-married,Craft-repair,Own-child,White,Male,0,0,40.0,United-States,<=50K
26,54.0,?,180211,Some-college,10,Married-civ-spouse,?,Husband,Asian-Pac-Islander,Male,0,0,NaN,South,>50K
27,39.0,Private,367260,HS-grad,9,Divorced,Exec-managerial,Not-in-family,White,Male,0,0,80.0,United-States,<=50K
28,49.0,Private,193366,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40.0,United-States,<=50K
29,23.0,Local-gov,190709,Assoc-acdm,12,Never-married,Protective-serv,Not-in-family,White,Male,0,0,52.0,United-States,<=50K
30,NaN,Private,266015,Some-college,10,Never-married,Sales,Own-child,Black,Male,0,0,44.0,United-States,<=50K
31,45.0,Private,386940,Bachelors,13,Divorced,Exec-managerial,Own-child,White,Male,0,1408,NaN,United-States,<=50K
32,NaN,Federal-gov,59951,Some-college,10,Married-civ-spouse,Adm-clerical,Own-child,White,Male,0,0,NaN,United-States,<=50K
33,22.0,State-gov,311512,Some-college,10,Married-civ-spouse,Other-service,Husband,Black,Male,0,0,15.0,United-States,<=50K
34,48.0,Private,242406,11th,7,Never-married,Machine-op-inspct,Unmarried,White,Male,0,0,40.0,Puerto-Rico,<=50K


##  Шкалирование данных

In [200]:
X_train, X_test, y_train, y_test = train_test_split(data, data.target, test_size=0.2,
                                                    random_state=0)
X_train.shape, X_test.shape

((26048, 15), (6512, 15))

Нормализация данных

In [37]:
ss = StandardScaler().fit(X_train[['Fare']])
X_train_copy = X_train.copy(deep=True)
X_train_copy['Fare_zscore'] = ss.transform(X_train_copy[['Fare']])
print(X_train_copy.head(6))

     Survived  Pclass     Sex   Age  SibSp     Fare  Fare_zscore
132         0       3  female  47.0      1  14.5000    -0.342483
97          1       1    male  23.0      0  63.3583     0.949028
22          1       3  female  15.0      0   8.0292    -0.513531
80          0       3    male  22.0      0   9.0000    -0.487869
101         0       3    male   NaN      0   7.8958    -0.517057
102         0       1    male  21.0      0  77.2875     1.317229


In [38]:
print(X_train_copy['Fare_zscore'].mean())
print(X_train_copy['Fare_zscore'].std())

-3.4918304806758956e-17
1.004056811789459


Мин-Макс шкалирование

In [39]:
mms = MinMaxScaler().fit(X_train[['Fare']])
X_train_copy = X_train.copy(deep=True)
X_train_copy['Fare_minmax'] = mms.transform(X_train_copy[['Fare']])
print(X_train_copy.head(6))

     Survived  Pclass     Sex   Age  SibSp     Fare  Fare_minmax
132         0       3  female  47.0      1  14.5000     0.030244
97          1       1    male  23.0      0  63.3583     0.220910
22          1       3  female  15.0      0   8.0292     0.004992
80          0       3    male  22.0      0   9.0000     0.008780
101         0       3    male   NaN      0   7.8958     0.004471
102         0       1    male  21.0      0  77.2875     0.275268


In [40]:
print(X_train_copy['Fare_minmax'].max())
print(X_train_copy['Fare_minmax'].min())

1.0
0.0


Робустное шкалирование

In [41]:
rs = RobustScaler().fit(X_train[['Fare']])
X_train_copy = X_train.copy(deep=True)
X_train_copy['Fare_robust'] = rs.transform(X_train_copy[['Fare']])
print(X_train_copy.head(6))

     Survived  Pclass     Sex   Age  SibSp     Fare  Fare_robust
132         0       3  female  47.0      1  14.5000     0.002050
97          1       1    male  23.0      0  63.3583     2.291278
22          1       3  female  15.0      0   8.0292    -0.301136
80          0       3    male  22.0      0   9.0000    -0.255649
101         0       3    male   NaN      0   7.8958    -0.307386
102         0       1    male  21.0      0  77.2875     2.943922


## Энкодинг переменных 

One-Hot encoding

In [42]:
data1 = pd.get_dummies(data,drop_first=True)

In [43]:
data1.head()

,Survived,Pclass,Age,SibSp,Fare,Sex_male
0,0,3,22.0,1,7.2500,1
1,1,1,38.0,1,71.2833,0
2,1,3,26.0,0,7.9250,0
3,1,1,35.0,1,53.1000,0
4,0,3,35.0,0,8.0500,1


Энкодинг значением

In [44]:
ord_enc = ce.OrdinalEncoder(cols=['Sex']).fit(X_train,y_train)

In [45]:
data4 = ord_enc.transform(data)
print(data4.head(5))

   Survived  Pclass  Sex   Age  SibSp     Fare
0         0       3    2  22.0      1   7.2500
1         1       1    1  38.0      1  71.2833
2         1       3    1  26.0      0   7.9250
3         1       1    1  35.0      1  53.1000
4         0       3    2  35.0      0   8.0500


Таргет энкодинг

In [46]:
target_enc = ce.TargetEncoder(cols=['Sex']).fit(X_train,y_train)

In [47]:
data2 = target_enc.transform(data)
data2.head()

,Survived,Pclass,Sex,Age,SibSp,Fare
0,0,3,0.164557,22.0,1,7.2500
1,1,1,0.755556,38.0,1,71.2833
2,1,3,0.755556,26.0,0,7.9250
3,1,1,0.755556,35.0,1,53.1000
4,0,3,0.164557,35.0,0,8.0500


WOE энкодинг

In [48]:
woe_enc = ce.WOEEncoder(cols=['Sex']).fit(X_train,y_train)

In [49]:
data3 = woe_enc.transform(data)
data3.head()

,Survived,Pclass,Sex,Age,SibSp,Fare
0,0,3,-1.088008,22.0,1,7.2500
1,1,1,1.548069,38.0,1,71.2833
2,1,3,1.548069,26.0,0,7.9250
3,1,1,1.548069,35.0,1,53.1000
4,0,3,-1.088008,35.0,0,8.0500


## Feature Selection

In [50]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
data = pd.DataFrame(np.c_[data['data'], data['target']],
                  columns= np.append(data['feature_names'], ['target']))

In [51]:
data.head(5)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0.0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0.0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0.0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0.0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0.0


In [52]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(labels=['target'], axis=1), 
                                                    data.target, test_size=0.2,
                                                    random_state=0)
X_train.shape, X_test.shape

((455, 30), (114, 30))

Константные значения

In [53]:
def constant_feature_detect(data,threshold=0.98):    
    data_copy = data.copy(deep=True)
    quasi_constant_feature = []
    for feature in data_copy.columns:
        predominant = (data_copy[feature].value_counts() / np.float(
                      len(data_copy))).sort_values(ascending=False).values[0]
        if predominant >= threshold:
            quasi_constant_feature.append(feature)
    print(len(quasi_constant_feature),'константные переменные')    
    return quasi_constant_feature

In [54]:
quasi_constant_feature = constant_feature_detect(data=X_train,threshold=0.9)

0 константные переменные


In [55]:
X_train['dummy'] = np.floor(X_train['worst smoothness']*10)
X_train.dummy.value_counts() / np.float(len(X_train))

1.0    0.923077
0.0    0.068132
2.0    0.008791
Name: dummy, dtype: float64

In [56]:
quasi_constant_feature = constant_feature_detect(data=X_train,threshold=0.9)
quasi_constant_feature

1 константные переменные


['dummy']

In [57]:
X_train.drop(labels=quasi_constant_feature,axis=1,inplace=True)
print(X_train.shape)

(455, 30)


Корреляционная фильтрация

In [58]:
def corr_feature_detect(data,threshold=0.8):
    
    corrmat = data.corr()
    corrmat = corrmat.abs().unstack() 
    corrmat = corrmat.sort_values(ascending=False)
    corrmat = corrmat[corrmat >= threshold]
    corrmat = corrmat[corrmat < 1] 
    corrmat = pd.DataFrame(corrmat).reset_index()
    corrmat.columns = ['feature1', 'feature2', 'corr']
   
    grouped_feature_ls = []
    correlated_groups = []
    
    for feature in corrmat.feature1.unique():
        if feature not in grouped_feature_ls:
    
            correlated_block = corrmat[corrmat.feature1 == feature]
            grouped_feature_ls = grouped_feature_ls + list(
                correlated_block.feature2.unique()) + [feature]
    
            correlated_groups.append(correlated_block)
    return correlated_groups

In [59]:
corr = corr_feature_detect(data=X_train,threshold=0.9)
for i in corr:
    print(i,'\n')

          feature1         feature2      corr
0   mean perimeter      mean radius  0.998185
6   mean perimeter        mean area  0.986692
14  mean perimeter  worst perimeter  0.970507
19  mean perimeter     worst radius  0.969520
33  mean perimeter       worst area  0.941920 

           feature1      feature2      corr
12  perimeter error  radius error  0.978323
30  perimeter error    area error  0.944995 

          feature1             feature2      corr
36  mean concavity  mean concave points  0.914627 

        feature1       feature2      corr
38  mean texture  worst texture  0.908182 

                feature1             feature2      corr
40  worst concave points  mean concave points  0.906312 



Взаимная информация

In [60]:
def mutual_info(X,y,select_k=10):

    if select_k >= 1:
        sel_ = SelectKBest(mutual_info_classif, k=select_k).fit(X,y)
        col = X.columns[sel_.get_support()]
        
    elif 0 < select_k < 1:
        sel_ = SelectPercentile(mutual_info_classif, percentile=select_k*100).fit(X,y)
        col = X.columns[sel_.get_support()]   
        
    else:
        raise ValueError("select_k должно быть положительным значением")
    
    return col

In [61]:
mi = mutual_info(X=X_train,y=y_train,select_k=3)
print(mi)

Index(['mean concave points', 'worst perimeter', 'worst area'], dtype='object')


In [62]:
mi = mutual_info(X=X_train,y=y_train,select_k=0.2)
print(mi)

Index(['mean perimeter', 'mean concave points', 'worst radius',
       'worst perimeter', 'worst area', 'worst concave points'],
      dtype='object')


Хи-квадрат тест

In [63]:
def chi_square_test(X,y,select_k=10):

    if select_k >= 1:
        sel_ = SelectKBest(chi2, k=select_k).fit(X,y)
        col = X.columns[sel_.get_support()]
    elif 0 < select_k < 1:
        sel_ = SelectPercentile(chi2, percentile=select_k*100).fit(X,y)
        col = X.columns[sel_.get_support()]   
    else:
        raise ValueError("select_k должно быть положительным значением")  
    
    return col

In [64]:
chi = chi_square_test(X=X_train,y=y_train,select_k=3)
print(chi)

Index(['mean area', 'area error', 'worst area'], dtype='object')


In [65]:
chi = chi_square_test(X=X_train,y=y_train,select_k=0.2)
print(chi)

Index(['mean perimeter', 'mean area', 'area error', 'worst radius',
       'worst perimeter', 'worst area'],
      dtype='object')


Одномерный ROC-AUC или MSE анализ

In [66]:
def univariate_roc_auc(X_train,y_train,X_test,y_test,threshold):

    roc_values = []
    for feature in X_train.columns:
        clf = DecisionTreeClassifier()
        clf.fit(X_train[feature].to_frame(), y_train)
        y_scored = clf.predict_proba(X_test[feature].to_frame())
        roc_values.append(roc_auc_score(y_test, y_scored[:, 1]))
    roc_values = pd.Series(roc_values)
    roc_values.index = X_train.columns
    print(roc_values.sort_values(ascending=False))
    print(len(roc_values[roc_values > threshold]), len(X_train.columns))
    keep_col = roc_values[roc_values > threshold]
    return keep_col

In [67]:
uni_roc_auc = univariate_roc_auc(X_train=X_train,y_train=y_train,
                                   X_test=X_test,y_test=y_test,threshold=0.8)
print(uni_roc_auc)

worst perimeter            0.917275
worst area                 0.895840
worst radius               0.893458
worst concave points       0.863131
mean concavity             0.856939
mean radius                0.849000
mean area                  0.839314
worst concavity            0.831375
mean perimeter             0.829628
mean concave points        0.826453
area error                 0.812321
worst compactness          0.742299
radius error               0.740235
mean compactness           0.734360
perimeter error            0.680534
worst texture              0.647666
worst fractal dimension    0.640997
concavity error            0.640203
worst symmetry             0.620991
concave points error       0.618133
compactness error          0.607336
mean symmetry              0.591775
mean texture               0.573357
texture error              0.568593
worst smoothness           0.565100
mean smoothness            0.557637
fractal dimension error    0.542077
smoothness error           0

In [68]:
def univariate_mse(X_train,y_train,X_test,y_test,threshold):

    mse_values = []
    for feature in X_train.columns:
        clf = DecisionTreeRegressor()
        clf.fit(X_train[feature].to_frame(), y_train)
        y_scored = clf.predict(X_test[feature].to_frame())
        mse_values.append(mean_squared_error(y_test, y_scored))
    mse_values = pd.Series(mse_values)
    mse_values.index = X_train.columns
    print(mse_values.sort_values(ascending=False))
    print(len(mse_values[mse_values > threshold]), len(X_train.columns))
    keep_col = mse_values[mse_values > threshold]
    return keep_col   

In [69]:
uni_mse = univariate_mse(X_train=X_train,y_train=y_train,
                            X_test=X_test,y_test=y_test,threshold=0.4)
print(uni_mse)

mean fractal dimension     0.491228
symmetry error             0.480750
fractal dimension error    0.456140
smoothness error           0.449561
texture error              0.412281
worst smoothness           0.403265
mean smoothness            0.399123
mean texture               0.396930
mean symmetry              0.363060
compactness error          0.361842
concave points error       0.357456
worst fractal dimension    0.355263
worst symmetry             0.350877
worst texture              0.333333
concavity error            0.333333
perimeter error            0.300439
mean compactness           0.258772
worst compactness          0.254386
radius error               0.245614
area error                 0.179825
mean perimeter             0.166667
mean concave points        0.166667
worst concavity            0.162281
mean radius                0.146930
mean concavity             0.142544
mean area                  0.140351
worst concave points       0.123782
worst area                 0

## Oversampling/undersampling 

In [70]:
len(y_train)

455

In [71]:
sm = SMOTE(random_state=42)

In [72]:
X_res, y_res = sm.fit_resample(X_train, y_train)

In [73]:
len(y_res)

580

In [74]:
cn = CondensedNearestNeighbour(random_state=42)

In [75]:
X_res, y_res = cn.fit_resample(X_train, y_train)

In [76]:
len(y_res)

205